<a href="https://colab.research.google.com/github/polinamaximenko/compling/blob/main/nmt_data_preprocessing_PM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

`Курс "Компьютерная лингвистика" | НИУ ВШЭ Санкт-Петербург 2024 (c) В.И. Фирсанова`

# Нейросетевой машинный перевод: подготовка данных к машинному обучению

План занятия:

I. Знакомимся с процессом подготовки данных к машинному обучению

II. Вместе проходим туториал TensorFlow и осваиваем модель кодер-декодер на примере рекуррентной нейросети

## Загрузка данных

In [ ]:
!wget https://raw.githubusercontent.com/vifirsanova/compling/main/data/toy_data.en
!wget https://raw.githubusercontent.com/vifirsanova/compling/main/data/toy_data.ru

--2024-02-12 11:42:04--  https://raw.githubusercontent.com/vifirsanova/compling/main/data/toy_data.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1590 (1.6K) [text/plain]
Saving to: ‘toy_data.en’

toy_data.en         100%[===================>]   1.55K  --.-KB/s    in 0s      

2024-02-12 11:42:04 (20.0 MB/s) - ‘toy_data.en’ saved [1590/1590]

--2024-02-12 11:42:04--  https://raw.githubusercontent.com/vifirsanova/compling/main/data/toy_data.ru
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2501 (2.4K) [text/plain]
Saving to: ‘

Импорт библиотек

In [ ]:
import numpy as np

from sklearn.model_selection import train_test_split

import tensorflow as tf

Запись данных в переменные

*X - данные, Y - метки*

*Seq2Seq*

*Seq_x -> encoder (рекур. нейросеть, к-я создает векторное представление текста) -> decoder (языковая модель, которая генерирует текст, основываясь на входных данных, рекур. нейросеть) -> Seq_y*

*Данные без выравнивания - мы хотим, чтобы модель сама определила соответствия - так перевод будет более естественным*

In [ ]:
def load_data(path):
  with open(path, 'r', encoding='utf-8') as f:
    return np.array(f.read().split('\n'))

In [ ]:
x_data, y_data = load_data('toy_data.ru'), load_data('toy_data.en')

print("Данные языка X:\n", x_data[5])
print("Данные языка Y:\n", y_data[5])

Данные языка X:
     Я люблю тебя.
Данные языка Y:
     I love you.


Создаем выборки

*Чтобы получить 3 выборки, делим на временную и train, затем временную делим на test и valid*

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data)

print("Тестовые данные языка X:\n", x_test)
print("Тестовые данные языка Y:\n", y_test)

Тестовые данные языка X:
 ['    Что ты делаешь?' '    Что у тебя в планах на будущее?'
 '    Какие у тебя хобби?' '    Я люблю тебя.' '' '    Привет!'
 '    Что ты обычно говоришь в таких ситуациях?'
 '    Как ты относишься к этому предложению?'
 '    Как ты относишься к этому вопросу?'
 '    Сколько времени у тебя занимает дорога до работы?'
 '    Какие у тебя планы на будущее?' '    Что ты будешь делать завтра?'
 '    Что ты любишь делать в свободное время?']
Тестовые данные языка Y:
 ['    What are you doing?' '    What are your plans for the future?'
 '    What are your hobbies?' '    I love you.' '' '    Hello!'
 '    What do you usually say in such situations?'
 '    What is your attitude toward this proposal?'
 '    What is your opinion on this matter?'
 '    How long does it take you to get to work?'
 '    What are your plans for the future?'
 '    What will you do tomorrow?'
 '    What do you like to do in your free time?']


## Кодирование

Создаем объекты Dataset

**Тензор** - объект векторного пространства V конечной размерности n

*shape* - размерность тензора

*dtype* - какие данные хранятся в объекте

**Батч** - некий отрезок данных; на батчи обычно делятся данные для машинного обучения

*shuffle* - случайное перемешивание, усложняет данные и затрудняет процесс обучения, таким образом получится более качественная модель, которая сможет легче адаптироваться к новым данным + избежать переобучения. Метод принимает на вход 1 аргумент, определяющий насколько сильно перемешаются данные (чем больше число, тем сильнее перемешивание).

Входные данные соотносятся с определенными метками, в их роли выступают тексты на целевом языке

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(64).shuffle(len(x_data))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(64).shuffle(len(x_data))

for x_strings, y_strings in train_dataset.take(1):
  print("Образец tf-датасета, данные языка X:\n", x_strings[1:3])
  print("\nОбразец tf-датасета, данные языка Y:\n", y_strings[1:3])

Образец tf-датасета, данные языка X:
 tf.Tensor(
[b'    \xd0\xa7\xd1\x82\xd0\xbe \xd1\x82\xd1\x8b \xd0\xbe\xd0\xb1\xd1\x8b\xd1\x87\xd0\xbd\xd0\xbe \xd0\xb5\xd1\x88\xd1\x8c \xd0\xbd\xd0\xb0 \xd0\xb7\xd0\xb0\xd0\xb2\xd1\x82\xd1\x80\xd0\xb0\xd0\xba?'
 b'    \xd0\xa7\xd1\x82\xd0\xbe \xd1\x82\xd1\x8b \xd1\x81\xd0\xbe\xd0\xb1\xd0\xb8\xd1\x80\xd0\xb0\xd0\xb5\xd1\x88\xd1\x8c\xd1\x81\xd1\x8f \xd0\xb4\xd0\xb5\xd0\xbb\xd0\xb0\xd1\x82\xd1\x8c \xd1\x81\xd0\xb5\xd0\xb3\xd0\xbe\xd0\xb4\xd0\xbd\xd1\x8f?'], shape=(2,), dtype=string)

Образец tf-датасета, данные языка Y:
 tf.Tensor(
[b'    What do you usually eat for breakfast?'
 b'    What are you going to do today?'], shape=(2,), dtype=string)


Лирическое отступление: чем матрица отличается от тензора?

Тензор - широкое понятие, в зависимости от его размерности выделяют:

- скалярные тензоры (хранит ровно одно значение); `shape ()`

- вектора (одномерный список значений любой длины);  `shape (n, )`

- матрицы (двумерный массив, как таблица); `shape (n, m)`

- тензоры (любая глубина); например `shape (n, m, k)` будет выглядеть как параллелепипед или как 3 состыкованные вместе матрицы (таблицы)

In [ ]:
print("Скалярный тензор с числом 42:\n", tf.constant(42))
print("\n Вектор длины n=2 с числами 42 и 24:\n", tf.constant([42, 24]))
print("\n Матрица длины содержит m=3 вектора с числами 42 и 24 длины n=2:\n", tf.constant([[42, 24],
                                                                                      [42, 24],
                                                                                      [42, 24]]))
print("\n Трёхмерный тензор содержит k=2 матрицы; \
матрица содержит m=3 вектора с числами 42 и 24 длины n=2:\n", tf.constant([[[42, 24], [42, 24], [42, 24]],
                                                                           [[42, 24], [42, 24], [42, 24]]]))
# NB! 32 бита - объем занимаемой памяти, типы данных можно менять

Скалярный тензор с числом 42:
 tf.Tensor(42, shape=(), dtype=int32)

 Вектор длины n=2 с числами 42 и 24:
 tf.Tensor([42 24], shape=(2,), dtype=int32)

 Матрица длины содержит m=3 вектора с числами 42 и 24 длины n=2:
 tf.Tensor(
[[42 24]
 [42 24]
 [42 24]], shape=(3, 2), dtype=int32)

 Трёхмерный тензор содержит k=2 матрицы; матрица содержит m=3 вектора с числами 42 и 24 длины n=2:
 tf.Tensor(
[[[42 24]
  [42 24]
  [42 24]]

 [[42 24]
  [42 24]
  [42 24]]], shape=(2, 3, 2), dtype=int32)


Теперь немного посчитаем:

In [ ]:
a = tf.constant([[1, 2],
                 [3, 4]])

b = tf.constant([[1, 1],
                 [1, 2]])

print("Сложение:\n", tf.add(a, b), "\n")
print("Умножение (поэлементное):\n", tf.multiply(a, b), "\n")
print("Умножение матриц:\n", tf.matmul(a, b), "\n")

# NB! К тензорам можно применять и другие функции, например, argmax

Сложение:
 tf.Tensor(
[[2 3]
 [4 6]], shape=(2, 2), dtype=int32) 

Умножение (поэлементное):
 tf.Tensor(
[[1 2]
 [3 8]], shape=(2, 2), dtype=int32) 

Умножение матриц:
 tf.Tensor(
[[ 3  5]
 [ 7 11]], shape=(2, 2), dtype=int32) 



Обычно используют прямоугольные тензоры, т.е. количество элементов в каждом компоненте должно совпадать, но бывают исключения, например, `RaggedTensor`.

In [ ]:
print('Без ragged мы получим ошибку!')

tf.ragged.constant([
                    ['this', 'is', 'a', 'sample', 'sentence'],
                    ['this', 'sentence', 'is', 'shorter']
                    ])

Без ragged мы получим ошибку!


<tf.RaggedTensor [[b'this', b'is', b'a', b'sample', b'sentence'],
 [b'this', b'sentence', b'is', b'shorter']]>

Тензоры бывают плотные `dense` и `разреженные`.

С разреженными векторами матрицами мы работаем, например, при использовании метода `one-hot encoding`. Такие данные занимают много памяти, их тяжело хранить в системе.

In [ ]:
print("Закодируем число 128 с помощью one-hot encoding\n")
sparse_tensor = tf.sparse.SparseTensor(indices=[[0, 1], [1, 2], [2, 8]],
                                       values=[1, 1, 1],
                                       dense_shape=[3, 10])

print("Метод sparse помогает нам сэкономить память компьютера", sparse_tensor, "\n")
print("Вот, как выглядит наша матрица на самом деле:\n", tf.sparse.to_dense(sparse_tensor))

Закодируем число 128 с помощью one-hot encoding

Метод sparse помогает нам сэкономить память компьютера SparseTensor(indices=tf.Tensor(
[[0 1]
 [1 2]
 [2 8]], shape=(3, 2), dtype=int64), values=tf.Tensor([1 1 1], shape=(3,), dtype=int32), dense_shape=tf.Tensor([ 3 10], shape=(2,), dtype=int64)) 

Вот, как выглядит наша матрица на самом деле:
 tf.Tensor(
[[0 1 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0]], shape=(3, 10), dtype=int32)


Паддинг - заполнение пустых слотов матрицы нулями

## Предобработка данных

Чистка и токенизация

In [ ]:
def tokenize(text):
  # к нижнему регистру
  text_lower = tf.strings.lower(text, encoding='utf-8')
  # оставим знаки препинания и все буквы латиницы и кириллицы
  text_clean = tf.strings.regex_replace(text_lower, '[^ a-zа-я.?!,]', '')
  # добавим пробел перед знаками препинания (для токенизации знаков)
  text_punct = tf.strings.regex_replace(text_clean, '[.?!,]', r' \0 ')
  # избавимся от лишних пробелов
  text = tf.strings.strip(text_punct)
  # добавим метки начала (sos: start of the sentence) и конца (eos: end of the sentence) предложений
  return tf.strings.join(['<SOS>', text, '<EOS>'], separator=' ')

In [ ]:
print("Образец токенизации:\n", tokenize(tf.constant("Как твои дела?")).numpy().decode())

Образец токенизации:
 <SOS> как твои дела ? <EOS>


Кодирование (векторизация): готовим данные для "скармливания" их модели

In [ ]:
x_text_processor = tf.keras.layers.TextVectorization(standardize=tokenize, ragged=True)

x_text_processor.adapt(train_dataset.map(lambda x, target: x))
print("Образец обучающего словаря языка X:\n", x_text_processor.get_vocabulary()[:10])
print("\nОбразец закодированного текста X:\n\tТекст:", x_strings[1].numpy().decode(), "\n\tКодирование:", x_text_processor(x_strings[1]))

y_text_processor = tf.keras.layers.TextVectorization(standardize=tokenize, ragged=True)

y_text_processor.adapt(train_dataset.map(lambda x, target: target))
print("\nОбразец обучающего словаря языка Y:\n", y_text_processor.get_vocabulary()[:10])
print("\nОбразец закодированного текста X:\n\tТекст:", y_strings[1].numpy().decode(), "\n\tКодирование:", y_text_processor(y_strings[1]))

Образец обучающего словаря языка X:
 ['', '[UNK]', '<SOS>', '<EOS>', '?', 'ты', 'что', 'как', 'твой', 'хочешь']

Образец закодированного текста X:
	Текст:     Что ты обычно ешь на завтрак? 
	Кодирование: tf.Tensor([ 2  6  5 65 82 21 79  4  3], shape=(9,), dtype=int64)

Образец обучающего словаря языка Y:
 ['', '[UNK]', '<SOS>', '<EOS>', '?', 'you', 'what', 'do', 'how', 'the']

Образец закодированного текста X:
	Текст:     What do you usually eat for breakfast? 
	Кодирование: tf.Tensor([ 2  6  7  5 40 82 26 86  4  3], shape=(10,), dtype=int64)


In [ ]:
def encode(x, y):
  # обработка текстов (текст -> словарь -> замена слов их ID'шками из словаря)
  x_encoded = x_text_processor(x).to_tensor()
  y_encoded = y_text_processor(y)
  # (x_encoded, y_encoded) -> ((x_encoded, y_input), y_label) для keras.Model.fit
  # Keras принимает (inputs, labels)
  # inputs = (x_encoded, y_input)
  # labels = y_label
  # y_label += следующий за y_input токен
  y_input = y_encoded[:,:-1].to_tensor()
  y_label = y_encoded[:,1:].to_tensor()
  return (x_encoded, y_input), y_label

In [ ]:
train = train_dataset.map(encode, tf.data.AUTOTUNE)
test = test_dataset.map(encode, tf.data.AUTOTUNE)

for (x_encoded_token, y_input_token), y_label_token in train.take(1):
  print("Закодированный текст x:")
  print(x_encoded_token[1, :10].numpy())
  print("\nЗакодированный текст y:")
  print(y_input_token[1, :10].numpy())
  print("\nТот же текст y мы сдвинули на 1 токен вперед:")
  print(y_label_token[1, :10].numpy())

Закодированный текст x:
[ 2  6  5 65 82 21 79  4  3  0]

Закодированный текст y:
[ 2  6  7  5 40 82 26 86  4  0]

Тот же текст y мы сдвинули на 1 токен вперед:
[ 6  7  5 40 82 26 86  4  3  0]


## Домашнее задание

Задание:

1. **По желанию**

* Ознакомиться с [туториалом TensorFlow](https://www.tensorflow.org/text/tutorials/nmt_with_attention) по машинному переводу

* Адаптировать препроцессинг на основе этого воркбука и обучить модель из туториала TensorFlow на данных `toy_data.ru` и `toy_data.en`

2. **Обязательно**

* Ниже: образец подсчета BLEU-Score для системы машинного перевода. Вспомните, как считается BLEU и опишите своими словами форумулу расчета и интерпретацию результатов. Ответьте на следующие вопросы:

  * Что значит кандидат и референс?
  * Для чего подойдет BLEU-Score: подсчета отдельных переводов или результатов работы системы на целом корпусе?
  * Какие "скоры" считаются высокими, а какие - низкими?
  
* Проанализируйте результаты в ячейках ниже. Порассуждайте:

  * На ваш взгляд, какие слова, контексты вызвали трудности у автоматической системы?
  * Руководствуясь знаниями о том, как работают нейросети, предположите, что вызвало ошибки.

In [ ]:
!pip install evaluate

In [ ]:
import evaluate

predictions = [
    "In addition to the usual hockey, there is an underwater and even ice version of this game.", #  Помимо обычного хоккея, существует подводная и даже подлёдная разновидности этой игры.
    "You reap what you sow.", # Что посеешь - то и пожнешь.
    "Also, justice is how we distribute the small resources." # Также, справедливость - это и то, как мы распределяем малые ресурсы.
    ]

references = [
    ["In addition to conventional hockey, there is an underwater and even under-ice variety of this game."], #  Помимо обычного хоккея, существует подводная и даже подлёдная разновидности этой игры.
    ["What goes around comes around."], # Что посеешь - то и пожнешь.
    ["Fairness also has to do with how we distribute scarce resources."], # Также, справедливость - это и то, как мы распределяем малые ресурсы.
    ]

bleu = evaluate.load("bleu")
results = bleu.compute(predictions=predictions, references=references)

print(results)

{'bleu': 0.374360558192794, 'precisions': [0.5833333333333334, 0.45454545454545453, 0.3333333333333333, 0.2222222222222222], 'brevity_penalty': 1.0, 'length_ratio': 1.0, 'translation_length': 36, 'reference_length': 36}


**Формула расчета** BLEU-Score

Общий принцип оценки: поиск пересечений n-грамм между машинным переводом и референсом

1. Вычисляется precision_1 (считаем встречаемость каждой униграммы в кандидате и референсе, выбираем из них минимум для каждой униграммы, складываем минимумы и делим на количество униграмм)
2. Вычисляется precision_2, precision_3, precision_4 с биграммами, триграммами, 4-граммами (*N-gram precisions*)
3. Вычисляется *Brevity Penalty*, штрафующий слишком короткие по сравнению с референсом переводы

!NB не оценивает смысловое содержание и грамматичность, не различает значимые и служебные слова, case sensitive, выбор токенизатора и регуляризации влияет на результат


  * Что значит кандидат и референс?

***Референс*** - эталонный перевод, золотой стандарт, с которым сравнивается машинный перевод; обычно включает несколько вариантов перевода одного текста

***Кандидат*** - машинный перевод, который мы хотим оценить

  * Для чего подойдет BLEU-Score: подсчета отдельных переводов или результатов работы системы на целом корпусе?

  Для подсчета результатов работы системы на целом корпусе. BLEU-Score - "корпусная" метрика, она нерепрезентативна на переводах отдельных предложений.

  * Какие "скоры" считаются высокими, а какие - низкими?


  BLEU-Score < 30: плохой перевод, содержащий множество ошибок

  BLEU-Score > 40: перевод высокого качества

  BLEU-Score > 50: перевод очень высокого качества (адекватный, беглый)

  BLEU-Score > 60: такой машинный перевод часто превосходит человеческий

  В данном примере BLEU-Score примерно равен 37, что расценивается как относительно хороший, понятный перевод (*understandable to good translations*)

  * На ваш взгляд, какие слова, контексты вызвали трудности у автоматической системы?

  Автоматической системе сложно перевести идиоматические выражения (в данном случае, пословица), а также слова, у которых много синонимичных переводческих соответствий (*обычный - usual/conventional, разновидность - variety/version, справедливость - fairness/justice* и др.) Порядок слов с *also* в третьем предложении тоже вызвал трудности, как и устойчивое выражение *has to do with ...*, прямого соответствия которому нет в русском. Забавно, что вместо *under-ice* модель перевела подледную разновидность как *ice*, хотя *ice version* - это и есть обычный хоккей.

  * Руководствуясь знаниями о том, как работают нейросети, предположите, что вызвало ошибки.

  Последовательность нейросети привела к дословному переводу идиоматического выражения "Что посеешь - то и пожнешь", хотя эталоном служит аналог с другой лексикой (*не учим выравнивание*). Здесь также прослеживается недостаток E2E, связанный с порядком слов - модель стремится копировать порядок исходного языка. Нейросетевая модель не совсем улавливает стилистические признаки (им можно обучить только на мегаданных), поэтому возникают трудности с выбором наиболее подходящего из переводческих соответствий.